In [1]:
from functions import get_data
import pandas as pd
import numpy as np


In [2]:
us_data=get_data('GOLD')

In [28]:
def x_points(data,index):
    # index=index.tolist()[0]
    val_h=data.iloc[index]
    start_date = val_h['mtime'].date()
    t=val_h['mtime'].time()
    filtered_data = data[(data['mtime'].dt.date == start_date) & (data.index <= index)]
    # print(filtered_data)
    # print(f'filtered_data:\n{filtered_data.head()}')
    max_value = filtered_data['Close'].max()
    min_value = filtered_data['Close'].min()
    return max_value, min_value
    
    

In [4]:
us_data['mtime']=pd.to_datetime(us_data['time'],unit='s')
us_data['mtime']

0        2015-01-02 08:00:00
1        2015-01-02 08:15:00
2        2015-01-02 08:30:00
3        2015-01-02 08:45:00
4        2015-01-02 09:00:00
                 ...        
211463   2023-12-15 22:45:00
211464   2023-12-15 23:00:00
211465   2023-12-15 23:15:00
211466   2023-12-15 23:30:00
211467   2023-12-15 23:45:00
Name: mtime, Length: 211468, dtype: datetime64[ns]

In [5]:
def distance_checker(price,value,lim):
    plh=price-value
    plh=abs(plh)
    if plh >lim:
        return 1
    else:
        return 0
    
    

In [44]:
def find_status(data,index,window,size):
    cspreads=[]
    # print(index,window,size)
    # print("fstatus here1")
    for i in range(window):
        datapoint=data.iloc[index-i]
        close=datapoint['Close']
        openv=datapoint['Open']
        cspread=abs(close-openv)
        cspreads.append(cspread)
    # print("fstatus here2")
    if max(cspreads)>size:
        return 1
    else:
        return 0
    

In [7]:
from functions import calc_prof

In [48]:
def daily(data,starttime,adjtime,lim,window,c_size,sl_adj,multiplier):
    trades=[]
    trade=0
    opendate=None
    openprice=0
    tp=0
    sl=0
    for i in range(len(data)):
        val_h=data.loc[i]
        time=val_h['mtime'].time()
        adj_time = pd.to_datetime(adjtime).time()# adtime being such astring '03:00:00'
        if time<adj_time:
            continue
        else:
            
            tday=val_h['mtime'].date()
            target_datetime = pd.to_datetime(f'{tday} {starttime}')
            # print(target_datetime)
            val=data[data['mtime'] == target_datetime]
            # print(val)
            # print(val.index)
            
            startindex=val.index
            ind=startindex.tolist()[0]
            begin_max_value,begin_min_value=x_points(data,ind)
            maxv=begin_max_value
            # print(maxv)
            minv=begin_min_value
            # print(minv)
            
            # print(f'tade{trade}')
            price=val_h['Close']
            if trade==0:
                if price>maxv:
                    if distance_checker(price,maxv,lim)==0:
                        maxv=price
                        continue
                    elif distance_checker(price,maxv,lim)==1:
                        # print('i am here 4')
                        if find_status(data,i,window,c_size)==1:
                            # print('HEREEE')
                            # print(ind)
                            trade=-1
                            openprice=price
                            opendate=val_h['time']
                            lowp=val_h['Low']
                            # print('i am here ')
                            sl=lowp-sl_adj
                            expectedloss=price-sl
                            target_tp=expectedloss*multiplier
                            tp=price+target_tp
                            # print(trade)

                            
                elif price<minv:
                    if distance_checker(price,minv,lim)==0:
                        minv=price
                        continue
                    elif distance_checker(price,minv,lim)==1:
                        # print('i am here3 ')
                        if find_status(data,i,window,c_size)==1:
                            # print('found me 1')
                            trade=1
                            openprice=price
                            # print(trade)
                            opendate=val_h['time']
                            # print('i am here2 ')
                            highp=val_h['High']
                            sl=highp-sl_adj
                            expectedloss=sl-price
                            target_tp=expectedloss*multiplier
                            tp=price-target_tp
            elif trade==1:
                # print('i am here8 ')
                if price>tp:
                    # print('i am here 6')
                    closedate=val_h['time'] 
                    profit=calc_prof(price,openprice)
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
                elif price<sl:
                    # print('i am here7 ')
                    closedate=val_h['time']
                    profit=calc_prof(price,openprice)
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
            elif trade==-1:
                # print('i am here 9 ')
                if price<tp:
                    # print('i am here 10 ')
                    closedate=val_h['time']
                    profit=calc_prof(openprice,price)
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
                elif price>sl:
                    # print('i am here 11 ')
                    closedate=val_h['time']
                    profit=calc_prof(openprice,price)
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
    return trades
                    
                

In [74]:
def daily(data,starttime,adjtime,lim,window,c_size,sl_adj,multiplier):
    trades=[]
    trade=0
    opendate=None
    openprice=0
    tp=0
    sl=0
    for i in range(len(data)):
        val_h=data.loc[i]
        time=val_h['mtime'].time()
        adj_time = pd.to_datetime(adjtime).time()# adtime being such astring '03:00:00'
        if time<adj_time:
            continue
        else:
            
            tday=val_h['mtime'].date()
            target_datetime = pd.to_datetime(f'{tday} {starttime}')
            # print(target_datetime)
            val=data[data['mtime'] == target_datetime]
            # print(val)
            # print(val.index)
            
            startindex=val.index
            ind=startindex.tolist()[0]
            begin_max_value,begin_min_value=x_points(data,ind)
            maxv=begin_max_value
            # print(maxv)
            minv=begin_min_value
            # print(minv)
            
            # print(f'tade{trade}')
            price=val_h['Close']
            if trade==0:
                if price>maxv:
                    if distance_checker(price,maxv,lim)==0:
                        maxv=price
                        continue
                    elif distance_checker(price,maxv,lim)==1:
                        # print('i am here 4')
                        if find_status(data,i,window,c_size)==1:
                            # print('HEREEE')
                            # print(ind)
                            trade=-1
                            openprice=price
                            opendate=val_h['time']
                            lowp=val_h['Low']
                            # print('i am here ')
                            sl=lowp-sl_adj
                            expectedloss=price-sl
                            target_tp=expectedloss*multiplier
                            tp=price+target_tp
                            # print(trade)

                            
                elif price<minv:
                    if distance_checker(price,minv,lim)==0:
                        minv=price
                        continue
                    elif distance_checker(price,minv,lim)==1:
                        # print('i am here3 ')
                        if find_status(data,i,window,c_size)==1:
                            # print('found me 1')
                            trade=1
                            openprice=price
                            # print(trade)
                            opendate=val_h['time']
                            # print('i am here2 ')
                            highp=val_h['High']
                            sl=highp-sl_adj
                            expectedloss=sl-price
                            target_tp=expectedloss*multiplier
                            tp=price-target_tp
            elif trade==1:
                # print('i am here8 ')
                if price>tp:
                    # print('i am here 6')
                    closedate=val_h['time'] 
                    profit=calc_prof(price,openprice)
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
                elif price<sl:
                    # print('i am here7 ')
                    closedate=val_h['time']
                    profit=calc_prof(price,openprice)
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
            elif trade==-1:
                # print('i am here 9 ')
                if price<tp:
                    # print('i am here 10 ')
                    closedate=val_h['time']
                    profit=calc_prof(openprice,price)
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
                elif price>sl:
                    # print('i am here 11 ')
                    closedate=val_h['time']
                    profit=calc_prof(openprice,price)
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
    return trades
                    
                

In [32]:
testing_dt=us_data2[:10000]



In [31]:
us_data2 = us_data[us_data['mtime'].dt.date != pd.to_datetime('2015-01-02').date()]


In [33]:
testing_dt.reset_index(drop=True, inplace=True)

In [23]:
val_h=testing_dt.loc[4000]
starttime='03:00:00'
tday=val_h['mtime'].date()
print(tday)
target_datetime = pd.to_datetime('2015-01-02 03:00:00')
print(target_datetime)
val=testing_dt[testing_dt['mtime'] == target_datetime]
print(val.index)
print(type(val.index))
pu=x_points(testing_dt,val.index)
pu

2015-03-04
2015-01-02 03:00:00
Index([], dtype='int64')
<class 'pandas.core.indexes.base.Index'>


IndexError: list index out of range

In [75]:
y=daily(testing_dt,'03:00:00','03:00:00',3.0,4,4.5,2.5,2.5)

In [50]:
from functions import evaluate

In [52]:
y

[]

In [51]:
ty=evaluate(y,'daily')

ValueError: attempt to get argmax of an empty sequence